In [1]:
import os
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
from pytz import timezone
from pprint import pprint

import cufflinks as cf
import plotly.graph_objects as go
import yfinance as yf


In [6]:
SPY_info_df = pd.read_csv('data/SPY-Info.csv')
SPY_df = pd.read_csv('data/SPY.csv', index_col='Date', parse_dates=True)

In [7]:
f = 'data/market_data/'

In [8]:
def get_first_dates():
    '''Get the earliest date for which market data is available for each company'''

    first_dates = []

    for file in os.listdir(f):
        df = pd.read_csv(f + file, index_col='Unnamed: 0', parse_dates=True)
        first_date = df.iloc[0].name
        ticker = file.split('.')[0]
        first_dates.append((ticker, first_date))
        first_dates = sorted(first_dates, key=lambda x: x[1])

    return first_dates

In [9]:
def make_combined_returns_df():
    '''Make dataframe of returns for all SPY stocks, as well as SPY index'''

    combined_returns = SPY_df.copy()
    combined_returns['Return'] = combined_returns['Close'].pct_change() * 100
    combined_returns.drop(
        ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'],
        axis=1, inplace=True
        )
    combined_returns.rename(columns={'Return': 'SPY'}, inplace=True)
    
    for file in os.listdir(f):
        ticker = file.replace('.csv', '')
        df = pd.read_csv(f + file, index_col='Unnamed: 0', parse_dates=True)
        df['Return'] = df['adjclose'].pct_change() * 100
        combined_returns = combined_returns.join(df['Return'], how='left')
        combined_returns.rename(columns={'Return': ticker}, inplace=True)

    return combined_returns

In [10]:
ticker_list = SPY_info_df['Symbol'].to_list()
sector_list = SPY_info_df['GICS Sector'].unique()
last_date = SPY_df.iloc[-1].name.date()
yr_ago = last_date - timedelta(days=365)
first_dates = get_first_dates()
combined_returns_df = make_combined_returns_df()

In [11]:
def get_weights():
    # Assign weights by sectors & sub-industries
    
    weights_df = pd.read_csv('data/SPY Weights.csv', index_col='Symbol')
    sector_weights = {}
    subIndustry_weights = {}
    ticker_weights = {}

    for sector in sector_list:
        subIndustry_list = SPY_info_df[SPY_info_df['GICS Sector'] == sector] \
                            ['GICS Sub-Industry'].unique()
        sector_weight = 0
        for subIndustry in subIndustry_list:
            tickers = SPY_info_df[SPY_info_df['GICS Sub-Industry'] == subIndustry] \
                        ['Symbol'].unique()
            subIndustry_weight = 0
            for ticker in tickers:
                weight = weights_df.loc[ticker, 'Weight']
                ticker_weights[ticker] = weight
                subIndustry_weight += weight

            subIndustry_weights[subIndustry] = subIndustry_weight
            sector_weight += subIndustry_weight
            
        sector_weights[sector] = sector_weight
       
    return sector_weights, subIndustry_weights, ticker_weights

sector_weights, subIndustry_weights, ticker_weights = get_weights()

In [12]:
print('Sector Weights:', sum(sector_weights.values()))
print('Sub-Industry Weights:', sum(subIndustry_weights.values()))
print('Ticker Weights:', sum(ticker_weights.values()))
# pprint(sector_weights)
# pprint(subIndustry_weights)
# pprint(ticker_weights)

Sector Weights: 99.990762
Sub-Industry Weights: 99.99076199999999
Ticker Weights: 99.990762


In [41]:
years = [yr for yr in range(first_dates[0][1].year, dt.now().year + 1)]
years[-2:]

[2021, 2022]

In [49]:
df = pd.read_csv(os.path.join(f, f'AAPL.csv'), index_col='Unnamed: 0')
df1 = pd.read_csv(os.path.join(f, f'ZTS.csv'), index_col='Unnamed: 0')
df = df.drop(index='period')
df1 = df1.drop(index='period')
df = df.loc[:, '2021'].astype('float')
df1 = df1.loc[:, '2021'].astype('float')
pd.concat([df, df1], axis=1)

,2021,2021
currentRatio,1.074553,3.856427
quickRatio,0.909660,2.568169
cashRatio,0.278449,1.937674
daysOfSalesOutstanding,51.390969,53.182227
daysOfInventoryOutstanding,11.276593,304.774208
operatingCycle,62.667561,357.956435
daysOfPayablesOutstanding,93.851071,69.101172
cashConversionCycle,-31.183510,288.855263
grossProfitMargin,0.417794,0.703832
operatingProfitMargin,0.297824,0.360468


In [20]:
SPY_df.value_counts?

In [54]:
def make_combined_annual_ratios_df(year='2021'):
    f = r'data\financial_ratios\Annual' 
    subIndustry_list = SPY_info_df['GICS Sub-Industry'].unique()  
    sector_ratios = {sector: {} for sector in sector_list}
    subIndustry_ratios = {si: {} for si in subIndustry_list}
    subIndustry_dfs = {si: pd.DataFrame() for si in subIndustry_list}
    ticker_ratios = {}
    weights = {}
#     pprint(weights)

    for sector in sector_list:
        sector_tickers = SPY_info_df[SPY_info_df['GICS Sector'] == sector] \
                         ['Symbol'].to_list()
        sector_dfs = pd.DataFrame()
        weights[sector] = []
        for ticker in sector_tickers:
            # Get sub-industry of ticker
            t_subIndustry = SPY_info_df[SPY_info_df['Symbol'] == ticker] \
                            ['GICS Sub-Industry'].item()
            weight = ticker_weights[ticker]
            ticker_sector_weight = weight / sector_weights[sector]
            ticker_subIndustry_weight = weight / subIndustry_weights[t_subIndustry]
            weights[sector].append(ticker_sector_weight)
            df = pd.read_csv(os.path.join(f, f'{ticker}.csv'), index_col='Unnamed: 0')
            df = df.drop(index='period').fillna(0)
            df = df.loc[:, year].astype('float')
            sector_df = df * ticker_sector_weight
            sector_dfs = pd.concat([sector_dfs, sector_df], axis=1)
            subIndustry_df = df * ticker_subIndustry_weight
            subIndustry_dfs[t_subIndustry] = pd.concat([subIndustry_dfs[t_subIndustry], 
                                                        subIndustry_df], axis=1)
            ticker_ratios[ticker] = df.to_dict()
        for ratio in df.index:
            sector_ratios[sector][ratio] = sum(sector_dfs.loc[ratio, :])
            for si in subIndustry_list:
                if not subIndustry_dfs[si].empty:
                    subIndustry_ratios[si][ratio] = sum(subIndustry_dfs[si].loc[ratio, :])
    #                 print(subIndustry_ratios[si])
    for si in sector_list:
        weights[si] = sum(weights[si])   
#         for si in subIndustry_ratios.keys():
#             print(si, subIndustry_ratios[si].shape, 
#                   SPY_info_df[SPY_info_df['GICS Sub-Industry'] == si]['Symbol'].value_counts().sum())    
#             print(si, subIndustry_ratios[si].shape)
#         print(sector_dfs.shape)        
#     pprint(weights)
#     pprint(sector_ratios)
    
#     pprint(subIndustry_ratios)
#     pprint(ticker_ratios)
    # pprint(ratios_yrs)
    return sector_ratios, subIndustry_ratios, ticker_ratios

sector_ratios, subIndustry_ratios, ticker_ratios = make_combined_annual_ratios_df()

In [56]:
pprint(sector_ratios)

{'Communication Services': {'assetTurnover': 0.5841229887022882,
                            'capitalExpenditureCoverageRatio': -3.795947556378646,
                            'cashConversionCycle': 0.973566868217532,
                            'cashFlowCoverageRatios': 2.1119444095944946,
                            'cashFlowToDebtRatio': 2.1258851570686104,
                            'cashPerShare': 100.01984386028384,
                            'cashRatio': 0.5251751191272669,
                            'companyEquityMultiplier': 1.919526139257946,
                            'currentRatio': 2.2987853613169404,
                            'daysOfInventoryOutstanding': 5.428204984074305,
                            'daysOfPayablesOutstanding': 57.01614161895842,
                            'daysOfSalesOutstanding': 53.05297105335102,
                            'debtEquityRatio': 0.9051454354385833,
                            'debtRatio': 0.4345950139731849,
                    

In [57]:
pprint(subIndustry_ratios)

{'Advertising': {'assetTurnover': 0.5078525914233388,
                 'capitalExpenditureCoverageRatio': -6.3027862723061805,
                 'cashConversionCycle': -143.8896384113787,
                 'cashFlowCoverageRatios': 0.3540013349700476,
                 'cashFlowToDebtRatio': 0.3540013349700476,
                 'cashPerShare': 17.591737132857475,
                 'cashRatio': 0.315629905482558,
                 'companyEquityMultiplier': 6.7055173990540595,
                 'currentRatio': 1.0027948038584036,
                 'daysOfInventoryOutstanding': 0.0,
                 'daysOfPayablesOutstanding': 383.01243835000435,
                 'daysOfSalesOutstanding': 239.1227999386257,
                 'debtEquityRatio': 5.695717000898579,
                 'debtRatio': 0.8460614871685042,
                 'dividendPaidAndCapexCoverageRatio': -11.043117766190218,
                 'dividendPayoutRatio': 0.4174148529149119,
                 'dividendYield': 0.034357204869747

             'companyEquityMultiplier': 2.0584925207384606,
             'currentRatio': 0.767130473193087,
             'daysOfInventoryOutstanding': 47.173361386377145,
             'daysOfPayablesOutstanding': 122.99543870356392,
             'daysOfSalesOutstanding': 31.2247439127601,
             'debtEquityRatio': 1.0400831774377473,
             'debtRatio': 0.5052644918353307,
             'dividendPaidAndCapexCoverageRatio': -4.198239060832444,
             'dividendPayoutRatio': 0.14696231480560806,
             'dividendYield': 0.014140931995106937,
             'ebitPerRevenue': 0.14148272809517787,
             'ebtPerEbit': 0.851897689768977,
             'effectiveTaxRate': 0.18603712671509282,
             'enterpriseValueMultiple': 7.457297723297994,
             'fixedAssetTurnover': 2.4519845434595937,
             'freeCashFlowOperatingCashFlowRatio': 0.6678741658722593,
             'freeCashFlowPerShare': 4.847324723247232,
             'grossProfitMargin': 0.3943

                                           'quickRatio': 0.8935751911184766,
                                           'receivablesTurnover': 5.4177486821206955,
                                           'returnOnAssets': 0.12873454994913283,
                                           'returnOnCapitalEmployed': 0.2502468325287004,
                                           'returnOnEquity': 0.5112303227937024,
                                           'shortTermCoverageRatios': 80.76858930421254,
                                           'totalDebtToCapitalization': 0.2402812392361614},
 'Distillers & Vintners': {'assetTurnover': 0.37192824315807804,
                           'capitalExpenditureCoverageRatio': -5.615555584347108,
                           'cashConversionCycle': 216.26649189461722,
                           'cashFlowCoverageRatios': 0.27174350747781845,
                           'cashFlowToDebtRatio': 0.27174350747781845,
                           'cashPerShare

 'Food Retail': {'assetTurnover': 2.7228227364267807,
                 'capitalExpenditureCoverageRatio': -2.3787085514834208,
                 'cashConversionCycle': 6.285793293962037,
                 'cashFlowCoverageRatios': 0.3310341477631515,
                 'cashFlowToDebtRatio': 0.3310341477631515,
                 'cashPerShare': 3.6002587322121604,
                 'cashRatio': 0.10978784329038137,
                 'companyEquityMultiplier': 5.081662489557226,
                 'currentRatio': 0.8136795522582324,
                 'daysOfInventoryOutstanding': 25.374715788852033,
                 'daysOfPayablesOutstanding': 23.995147494512633,
                 'daysOfSalesOutstanding': 4.906224999622636,
                 'debtEquityRatio': 4.0843776106934,
                 'debtRatio': 0.803748304631951,
                 'dividendPaidAndCapexCoverageRatio': -2.9236379236379237,
                 'dividendPayoutRatio': 0.20657640232108315,
                 'dividendYield': 0.02

                        'debtEquityRatio': 18.859135911409417,
                        'debtRatio': 0.809165321050662,
                        'dividendPaidAndCapexCoverageRatio': 3.0686136345912294,
                        'dividendPayoutRatio': 0.4856297231264167,
                        'dividendYield': 0.014987682869954111,
                        'ebitPerRevenue': 0.24119828697507717,
                        'ebtPerEbit': 0.6819066484822595,
                        'effectiveTaxRate': 0.09646686099202272,
                        'enterpriseValueMultiple': 22.937192868848264,
                        'fixedAssetTurnover': 4.436384670448856,
                        'freeCashFlowOperatingCashFlowRatio': 0.8765598024196533,
                        'freeCashFlowPerShare': 5.213397986445486,
                        'grossProfitMargin': 0.46990546555797286,
                        'interestCoverage': -5.552900611700053,
                        'inventoryTurnover': 5.046407095908277,
     

                                    'cashConversionCycle': 0.0,
                                    'cashFlowCoverageRatios': 0.07368994410256419,
                                    'cashFlowToDebtRatio': 0.07368994410256419,
                                    'cashPerShare': 300.07310107864987,
                                    'cashRatio': 4.412916250797567,
                                    'companyEquityMultiplier': 11.873132221980626,
                                    'currentRatio': 9.937145822969015,
                                    'daysOfInventoryOutstanding': 0.0,
                                    'daysOfPayablesOutstanding': 0.0,
                                    'daysOfSalesOutstanding': 1051.9583980540287,
                                    'debtEquityRatio': 10.869480299142147,
                                    'debtRatio': 0.9139919244188035,
                                    'dividendPaidAndCapexCoverageRatio': 6.606392178902597,
                    

                                    'effectiveTaxRate': 0.36284636432151113,
                                    'enterpriseValueMultiple': 6907.675721994623,
                                    'fixedAssetTurnover': 3.545118302533628,
                                    'freeCashFlowOperatingCashFlowRatio': 0.7142167648598668,
                                    'freeCashFlowPerShare': 0.9458616676017264,
                                    'grossProfitMargin': 0.16143953446002382,
                                    'interestCoverage': 2.1889736983748227,
                                    'inventoryTurnover': 5.4173339046401106,
                                    'longTermDebtToCapitalization': 0.2194429663298194,
                                    'netIncomePerEBT': 0.5982135631980928,
                                    'netProfitMargin': 0.06517379474687351,
                                    'operatingCashFlowPerShare': 1.360604419003211,
                                    

                'freeCashFlowPerShare': 1.435357902000199,
                'grossProfitMargin': 0.0,
                'interestCoverage': 11.18867924528302,
                'inventoryTurnover': 0.0,
                'longTermDebtToCapitalization': 0.2928866689631416,
                'netIncomePerEBT': 0.7333333333333333,
                'netProfitMargin': 0.03526394528745459,
                'operatingCashFlowPerShare': 2.0962664991431477,
                'operatingCashFlowSalesRatio': 0.13218636460782218,
                'operatingCycle': 0.0,
                'operatingProfitMargin': 0.06336824107715323,
                'payablesTurnover': 0.0,
                'payoutRatio': 0.493939393939394,
                'pretaxProfitMargin': 0.048087198119256255,
                'priceBookValueRatio': 1.7505906339549946,
                'priceCashFlowRatio': 11.620129098952676,
                'priceEarningsRatio': 43.557877864862,
                'priceEarningsToGrowthRatio': 0.3459007948091982,


                       'fixedAssetTurnover': 2.5814264680084786,
                       'freeCashFlowOperatingCashFlowRatio': 0.642260793780193,
                       'freeCashFlowPerShare': 5.782648965016607,
                       'grossProfitMargin': 0.6895659379140496,
                       'interestCoverage': 4.68155653598156,
                       'inventoryTurnover': 0.676357165478712,
                       'longTermDebtToCapitalization': 0.7530772315232702,
                       'netIncomePerEBT': 0.9022683570172905,
                       'netProfitMargin': 0.26772717200635343,
                       'operatingCashFlowPerShare': 11.647326523073918,
                       'operatingCashFlowSalesRatio': 0.4825908751959072,
                       'operatingCycle': 37.727217840144256,
                       'operatingProfitMargin': 0.3042679555070999,
                       'payablesTurnover': 11.667541923937874,
                       'payoutRatio': 1.2441851079436985,
     

In [10]:
def make_combined_annual_ratios_df(year='2021'):
    f = r'data\financial_ratios\Annual'
    df = pd.read_csv(os.path.join(f, os.listdir(f)[0]), 
                     skiprows=[1], index_col='Unnamed: 0')
    # print(df)
    ratios = df.index.to_list()
    # years = [yr for yr in range(first_dates[0][1].year, dt.now().year + 1)]
    subIndustry_list = SPY_info_df['GICS Sub-Industry'].unique()  
    sector_ratios = {ratio: {sector: [] for sector in sector_list} 
                     for ratio in ratios}
    subIndustry_ratios = {ratio: {si: [] for si in subIndustry_list} 
                          for ratio in ratios}
    ticker_ratios = {ratio: {} for ratio in ratios}
    # ratios_yrs = {yr: {} for yr in years}

    # for year in years[-3:]:
    for ratio in ratios:    
        for sector in sector_list:
            sector_tickers = SPY_info_df[SPY_info_df['GICS Sector'] == sector] \
                            ['Symbol'].to_list() 
            for ticker in sector_tickers:
                # Get sub-industry of ticker
                t_subIndustry = SPY_info_df[SPY_info_df['Symbol'] == ticker] \
                                ['GICS Sub-Industry'].item()
                df = pd.read_csv(os.path.join(f, f'{ticker}.csv'), 
                                 index_col='Unnamed: 0')
                df = df.drop(index='period')
                # years = df.columns
                weight = ticker_weights[ticker]
                f_ratio = float(df.loc[ratio, year])

                if math.isnan(f_ratio):
                    f_ratio = 0
                
                weighted_ratio = f_ratio * weight
                
            # print(sector, weighted_ratio)
                sector_ratios[ratio][sector].append(weighted_ratio)
            
            sector_ratios[ratio][sector] = sum(sector_ratios[ratio][sector])
        # print(ratio)
    
    pprint(sector_ratios)
    # pprint(subIndustry_ratios)
    # pprint(ticker_ratios)
    # pprint(ratios_yrs)

    

make_combined_annual_ratios_df()
            

KeyError: 'assetTurnover'

In [12]:
df = pd.read_csv('data/financial_ratios/Annual/AAPL.csv', skiprows=[1], index_col='Unnamed: 0')
df.index.to_list()
df.head()

,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
currentRatio,1.074553,1.363604,1.540126,1.123843,1.276063,1.352669,1.108771,1.080113,1.678639,1.495849,...,2.302340,1.729665,2.495091,2.353328,2.339825,2.563128,2.155985,2.728601,3.164134,2.786441
quickRatio,0.909660,1.218195,1.384447,0.986566,1.089670,1.220844,0.892495,0.824234,1.401507,1.240750,...,1.460266,0.906699,1.769285,1.479047,1.712756,1.789944,1.779927,2.027140,2.550152,1.888136
cashRatio,0.278449,0.360710,0.462022,0.221733,0.201252,0.259271,0.262002,0.218194,0.326607,0.278813,...,0.618981,0.269537,0.349930,0.496302,0.365141,0.489385,0.449819,1.179541,1.750760,1.142373
daysOfSalesOutstanding,51.390969,49.787534,64.258765,67.332499,56.800671,49.592768,47.387609,54.350146,44.081476,43.592532,...,62.815049,63.235552,55.983491,52.473451,50.041382,54.777631,41.332596,55.689590,50.470557,41.866528
daysOfInventoryOutstanding,11.276593,8.741883,9.263639,8.817631,12.563631,5.923304,6.120288,6.863787,6.039623,3.286604,...,58.039933,104.792341,53.044350,74.013277,49.721796,64.332096,16.315922,63.649691,44.652076,54.521467


In [2]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise

In [3]:
# get historical market data
hist = msft.history(period="max")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.055536,0.063703,0.055536,0.060980,1031788800,0.00,0.0
1986-03-14,0.060980,0.064247,0.060980,0.063158,308160000,0.00,0.0
1986-03-17,0.063158,0.064792,0.063158,0.064247,133171200,0.00,0.0
1986-03-18,0.064247,0.064792,0.062069,0.062613,67766400,0.00,0.0
1986-03-19,0.062613,0.063158,0.060980,0.061524,47894400,0.00,0.0
...,...,...,...,...,...,...,...
2022-08-16,291.371520,293.417196,289.804869,292.089996,18102900,0.00,0.0
2022-08-17,289.739990,293.350006,289.470001,291.320007,18253400,0.62,0.0
2022-08-18,290.190002,291.910004,289.079987,290.170013,17186200,0.00,0.0


In [5]:
# show financials
# msft.financials
msft.quarterly_financials


,2022-06-30,2022-03-31,2021-12-31,2021-09-30
Research Development,6849000000.0,6306000000.0,5758000000.0,5599000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,20487000000.0,20190000000.0,22515000000.0,20524000000.0
Minority Interest,None,None,None,None
Net Income,16740000000.0,16728000000.0,18765000000.0,20505000000.0
Selling General Administrative,8053000000.0,7075000000.0,6763000000.0,5834000000.0
Gross Profit,35436000000.0,33745000000.0,34768000000.0,31671000000.0
Ebit,20534000000.0,20364000000.0,22247000000.0,20238000000.0
Operating Income,20534000000.0,20364000000.0,22247000000.0,20238000000.0
Other Operating Expenses,None,None,None,None


In [ ]:
# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits


# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show all earnings dates
msft.earnings_dates

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# show news
msft.news